In [1]:
!pip install tiktoken
!pip install langchain==0.1.7
!pip install pypdf
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need 

In [2]:
!pip install faiss-cpu
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [3]:
import os
import openai
import tiktoken
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA

In [4]:
import os

os.environ["OPENAI_API_KEY"] = '45462e152b294e11bbcd89f632b6a4df'
os.environ["OPENAI_API_TYPE"] = 'azure'
os.environ["OPENAI_API_VERSION"] = '2023-05-15'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://sktflyailab.openai.azure.com/'

### 토크나이저 객체 생성

In [5]:
import tiktoken
# 토크 나이저 객체 생성
tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)


### 스플리터 만들기

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=tiktoken_len,
)

### 데이터 불러오기

In [7]:
loader = PyPDFLoader('/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf')
pages = loader.load_and_split(text_splitter)

texts = text_splitter.split_documents(pages)

### 임베딩

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 데이터 베이스 생성

In [9]:
db = FAISS.from_documents(texts, hf)

In [10]:
chatgpt = AzureChatOpenAI(
 deployment_name = 'dev-gpt-35-turbo',
 temperature=0.2,

)

In [11]:
query ='탄소중립 사회란?'

### 설명 찾아보기..

In [14]:
qa = RetrievalQA.from_chain_type(llm=chatgpt,
                                 chain_type="stuff",
                                 retriever=db.as_retriever(
                                     search_type="similarity",
                                     search_kwargs={"k": 3}
                                 ))

In [15]:
qa(query)

{'query': '탄소중립 사회란?',
 'result': "탄소중립 사회는 개인, 회사, 단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 '순배출이 제로(0)'가 되게 하는 개념입니다. 이는 인간의 활동에 의한 온실가스 배출을 최대한 줄이고, 남은 온실가스는 흡수, 제거하여 순배출이 제로가 되는 것을 의미합니다. 탄소중립은 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있습니다."}

In [16]:
from re import search
qa = RetrievalQA.from_chain_type(
    llm=chatgpt,
    chain_type='stuff',
    retriever=db.as_retriever(
        search_type='similarity',
        search_kwargs={'k': 3, 'fetch_k':10}
    ),return_source_documents=True
)
qa(query) # result : 요약된 결과
# source_documents : 원문 내용

{'query': '탄소중립 사회란?',
 'result': "탄소중립 사회는 개인, 회사, 단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 '순배출이 제로(0)'가 되게 하는 개념입니다. 이는 인간의 활동에 의한 온실가스 배출을 최대한 줄이고, 남은 온실가스는 흡수, 제거하여 순배출이 제로가 되는 것을 의미합니다. 탄소중립은 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있습니다.",
 'source_documents': [Document(page_content='01이노비즈 정책브리프\n01 탄소중립과 대응 필요성\n탄소중립이란 ?\n∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제\n로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함\n- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로\n(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음\n*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수\n소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 \n대응 필요성\n∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가\n운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨', metadata={'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf', 'page': 3}),
  Document(page_content='보제공 확대 \n- 재생원료 사용 극대화 , 철강, 플라스틱을 대체하는 혁신소재 개발, 탄소발자국 * 등 친환경 제품 정보 제\n공 확대 등 순환 사회로의 전환 가속화\n*탄소발자국이란 개인 